In [1]:
import numpy as np
import keras
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Dropout
from keras_contrib.layers.crf import CRF
import matplotlib.pyplot as plt
import os
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from keras_contrib.layers.crf import CRF, crf_loss, crf_viterbi_accuracy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

Using TensorFlow backend.


In [2]:
#调用模型，‘’内为模型路径
model1_0 = keras.models.load_model('/home/wcb/nerXY/model/model1.0.h5', custom_objects={"CRF": CRF, 'crf_loss': crf_loss,'crf_viterbi_accuracy': crf_viterbi_accuracy})




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-31
OMP: Info #156: KMP_AFFINITY: 32 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 1 packages x 16 cores/pkg x 2 threads/core (16 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 core 0 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 16 maps to package 0 core 0 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 1 maps to package 0 core 1 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 17 maps to package 0 core 1 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 2 maps to package 0 core 2 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 18 maps to package 0 core 2 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 3 maps to package 0 core 3 thread 0 
OMP: Info #171: KMP_AFFI

In [3]:
#读入模型对应的词典
word_dict = {}
file = open('/home/wcb/nerXY/word_dict/1.0_word_dict.txt','r')
for line in file.readlines():
    line = line.strip()
    v = line.split(' ')[0]
    k = line.split(' ')[1]
    word_dict[k] = v

file.close()
#print(word_dict)

In [13]:
#导入标签列表，不同模型的识别标签不同，标签列表顺序应与模型训练时的标签列表顺序一致
class_dict = {'O':0,
              'I-MC':1,
              'I-PCC':2,
              'I-DTS':3
             }

In [7]:
#获取模型预测标签与真实标签
x_pre = []
y_true = []
text = []
sen = []
sen_word_tag = []
tag_true = []
#读取测试数据
for line in open('/home/wcb/nerXY/test_data/test1.0.txt','r',encoding='utf-8'):
    line = line.rstrip().split('\t')
    if not line:
        continue
    char = line[0]
    if not char:
        continue
    if char not in word_dict:
        char = 'UNK'
    tag = line[-1]
    sen.append(char)
    sen_word_tag.append(word_dict.get(char))
    tag_true.append(tag)
    if char == '.':
        sen_word_tag = pad_sequences([sen_word_tag], maxlen=40, padding='post')
        x_pre.append(sen_word_tag)
        text.append(sen)
        y_true.append(tag_true)
        sen_word_tag = []
        sen = []
        tag_true = []
#print(x_pre)
#print(text)
#print(y_true)

In [16]:
#获取预测标签
raw = []
for i in range(0,len(x_pre)):
    raw.append(model1_0.predict(x_pre[i])[0][-40:])
label_dict = {j:i for i,j in class_dict.items()}
result = []
for i in range(0,len(raw)):
    result.append([np.argmax(row) for row in raw[i]])

chars = [i for i in text]

tags = []
for j in range(0,len(result)):
    tags.append([label_dict.get(i) for i in result[j]][:len(text[j])])

y_pre = []
for i in range(0,len(result)):
    y_pre.append(list(zip(chars[i], tags[i])))

for i in range(0,len(y_true)):
    if len(y_true[i]) > 40:
        y_true[i] = y_true[i][40]

In [17]:
#计算精度，请注意输入匹配的模型与数据
TP = 0
FP = 0
FN = 0
for i in range(0,len(y_true)):
    for j in range(0,len(y_true[i])):
        if y_true[i][j] not in ['O']:
            if y_true[i][j] == y_pre[i][j][1]:
                TP = TP + 1
            else:
                FN = FN + 1
        else:
            if y_pre[i][j][1] not in ['O']:
                FP = FP + 1
print(TP,FP,FN)
P = TP/(TP+FP)
R = TP/(TP+FN)
F = (2*P*R)/(P+R)
print(P,R,F)

0 277 1775


ZeroDivisionError: float division by zero